<img src="assets/logos/Logo_endpoint.png" width=500></img>


## Flujo de la implementación
<img src="assets/build/deployment-flow.png" width=800></img>



## Configuración de pipeline
Tus pipelines son configurados por dos archivos, shoot-config.json y layout-config.json. Puedes editar cosas como los hiperparámetros, las variables ambientales de implementación y los tipos de instancias. A continuación se describen las propiedades que cambias o debes cambiar. 

|Nombre del parámetro| Descripción | Valor sugerido|
|---|---|---|
|frameworkversion|versión del marco de trabajo|"1.3.0"|
|inputmode|"Archivo" o "Tubería" según qué modo de entrada desees|"File"|
|hyperparameters|hiperparámetros de valores clave para enviar a tu trabajo de entrenamiento|{}|
|channels|los canales de datos para la instancia, el valor sugerido creará un canal de datos de entrenamiento que apuntará a datos con el prefijo "data/train" en el bucket de datos|{"train":{"path":"data/train"}}|
|trainvolumesize|qué cantidad de almacenamiento EBS adjuntar a las instancias de entrenamiento|"5"|
|traininstancecount|cuántas instancias deseas usar para la capacitación|1|
|hostinstancetype|inference instance type|"ml.t2.medium"|
|traininstancetype|tipo de instancia de entrenamiento|"ml.m5.large"|

para los detalles sobre posibles parámetros y valores, consulta la [documentación aws-sagemaker-build](https://github.com/aws-samples/aws-sagemaker-build#Parameters). Cuando ejecutes el script startPipeline.js, estos archivos de configuración se fusionarán en el almacén de parámetros SSM que configura los pipelines.


## Probar el punto final implementado
para probar el punto final (tu modelo implementado) ejecuta:
- (opcionalmente) edita /bin/testLayout.py o /bin/testShoot.py para configurar/personalizar tu prueba
- ejecuta 
```shell
/bin/testLayout.py # test for Defensive team
/bin/testShoot.py # test for Offensive team
```

## Aplicar tus cambios
una vez que hayas hecho tus cambios y estés listo para implementarlos en la producción, sigue los pasos a continuación
1. ejecuta el siguiente comando para aplicar los archivos de la etapa que cambies
```shell
git add <your file>
```
1. aplica tus cambios con un mensaje
```shell
git commit -m "<your message>"
```
1. aplica tus cambios en el repositorio remoto
```shell
git push origin shoot #for offense
git push origin layout # for defense
```
1. informa a tus operaciones que hay cambios para implementar

## Datos
Todos los juegos se graban y están disponibles en el bucket de archivos de juegos. No podrás acceder al bucket de datos hasta que el grupo de operaciones haya inscrito a tu equipo.

### Descargar
El siguiente script descargará todos los datos del bucket S3 (configurado en config.js), descomprímelo y fusiónalo en un solo archivo en /data/data.json
```shell
./bin/download_data.sh
```

### Formato:
El archivo /data/data.json es un archivo json nuevo con una línea eliminada con el siguiente formato
```json
{                                                                                      
    "Status":"Finished",
    "winner":"B",
    "start_time":"<epoch time>",
    "end_time":"<epoch time>",
	"GameId":"d0d73e64-bd47-4bb2-ab50-ed0bd1935066",
    "TeamA":{
        "layout":[[0,0 ...],...],
        "score":0,
        "session":{},
        "name":"purple people eaters",
		"shots":[{"x":6,"y":1},....]
     },
     "TeamB":{
        "layout":[[0,0 ...],...],
        "score":0,
        "session":{},
        "name":"the dr strange loves",
		"shots":[{"x":1,"y":4},....]
	 }
}
```
|Campo de nivel superior|Descripción|
|---|---|
|Status| Finalizado o Fallido, dependiendo del estado del juego|
|winner| el nombre del equipo ganador|
|start_time| cuándo comenzó el juego|
|end_time| cuándo terminó el juego|
|GameId| Un ID único del juegoPuede ser usado con la página web de replay para ver un juego específico|

|Campo del Equipo*|Descripción|
|---|---|
|layout|la distribución de barcos en este juego|
|score| cuántos casilleros de barcos fueron acertados por este equipo|
|session| el último estado de la sesión en este juego para este equipo|
|shots| una lista ordenada de disparos para este juego. disparos[0] es el primer disparo, disparos[1] es el segundo etc.|
|name| el nombre de equipo para este equipo|

### Cargar datos de entrenamiento
después de haber hecho cualquier modificación a los datos, deberás cargarlos al bucket de datos correspondiente ejecutando lo siguiente
```shell
./bin/upload_data.json
```

## Transferir y aplicar
Cuando tengas un nuevo código escrito y terminado para entrenar e implementar tu modelo, aplica los cambios a tu módulo git e indica al equipo de operaciones que incorpore tus cambios e implemente tu código.

```shell
git commit -a -m "I am adding these features <these features>"
git push -u origin $(git branch-name)
```